In [2]:
from argparse import Namespace

In [6]:
import argparse
from argparse import Namespace

import torch
import torch.nn as nn
import torch.optim as optim

from modules.trainer import Trainer
from modules.data_loader import DataLoader

from modules.models.rnn import RNNClassifier
from modules.models.cnn import CNNClassifier



def main(config):
    loaders = DataLoader(
        train_fn=config.train_fn,
        batch_size=config.batch_size,
        min_freq=config.min_vocab_freq,
        max_vocab=config.max_vocab_size,
        device=config.gpu_id
    )

    print(
        '|train| =', len(loaders.train_loader.dataset),
        '|valid| =', len(loaders.valid_loader.dataset),
    )
    
    vocab_size = len(loaders.text.vocab)
    n_classes = len(loaders.label.vocab)
    print('|vocab| =', vocab_size, '|classes| =', n_classes)

    if config.rnn is False and config.cnn is False:
        raise Exception('You need to specify an architecture to train. (--rnn or --cnn)')

    if config.rnn:
        # Declare model and loss.
        model = RNNClassifier(
            input_size=vocab_size,
            word_vec_size=config.word_vec_size,
            hidden_size=config.hidden_size,
            n_classes=n_classes,
            n_layers=config.n_layers,
            dropout_p=config.dropout,
        )
        optimizer = optim.Adam(model.parameters())
        crit = nn.NLLLoss()
        print(model)

        if config.gpu_id >= 0:
            model.cuda(config.gpu_id)
            crit.cuda(config.gpu_id)

        rnn_trainer = Trainer(config)
        rnn_model = rnn_trainer.train(
            model,
            crit,
            optimizer,
            loaders.train_loader,
            loaders.valid_loader
        )
    if config.cnn:
        # Declare model and loss.
        model = CNNClassifier(
            input_size=vocab_size,
            word_vec_size=config.word_vec_size,
            n_classes=n_classes,
            use_batch_norm=config.use_batch_norm,
            dropout_p=config.dropout,
            window_sizes=config.window_sizes,
            n_filters=config.n_filters,
        )
        optimizer = optim.Adam(model.parameters())
        crit = nn.NLLLoss()
        print(model)

        if config.gpu_id >= 0:
            model.cuda(config.gpu_id)
            crit.cuda(config.gpu_id)

        cnn_trainer = Trainer(config)
        cnn_model = cnn_trainer.train(
            model,
            crit,
            optimizer,
            loaders.train_loader,
            loaders.valid_loader
        )

    torch.save({
        'rnn': rnn_model.state_dict() if config.rnn else None,
        'cnn': cnn_model.state_dict() if config.cnn else None,
        'config': config,
        'vocab': loaders.text.vocab,
        'classes': loaders.label.vocab,
    }, config.model_fn)

In [20]:
config = {
    'model_fn': './moedels/review.pth',
    'train_fn': './data/review.sorted.uniq.refined.tok.shuf.train.tsv',
    'gpu_id': 0,
    'verbose': 2,
    'min_vocab_freq': 5,
    'max_vocab_size': 999999,
    'batch_size': 16,
    'n_epochs': 2,
    'word_vec_size': 256,
    'dropout': .3,
    'max_length': 256,
    'rnn': True,
    'hidden_size': 512,
    'n_layers': 4,
    'cnn': True,
    'use_batch_norm': True,
    'window_sizes': [3, 4, 5],
    'n_filters': [100, 100, 100]
}

config = Namespace(**config)

print(config)
main(config)

Namespace(model_fn='./moedels/review.pth', train_fn='./data/review.sorted.uniq.refined.tok.shuf.train.tsv', gpu_id=0, verbose=2, min_vocab_freq=5, max_vocab_size=999999, batch_size=32, n_epochs=2, word_vec_size=256, dropout=0.3, max_length=256, rnn=True, hidden_size=512, n_layers=4, cnn=True, use_batch_norm=True, window_sizes=[3, 4, 5], n_filters=[100, 100, 100])
|train| = 118314 |valid| = 29579
|vocab| = 12957 |classes| = 2
RNNClassifier(
  (emb): Embedding(12957, 256)
  (rnn): LSTM(256, 512, num_layers=4, batch_first=True, dropout=0.3, bidirectional=True)
  (generator): Linear(in_features=1024, out_features=2, bias=True)
  (activation): LogSoftmax(dim=-1)
)
Epoch [1/2]:   2%|      | 66/3698 [00:07<05:57, 10.16it/s, accuracy=0.722, loss=0.555, |g_param|=0.637, |param|=1.82e+3]Current run is terminating due to exception: CUDA out of memory. Tried to allocate 742.00 MiB (GPU 0; 3.00 GiB total capacity; 1.02 GiB already allocated; 635.55 MiB free; 1.33 GiB reserved in total by PyTorch)
E

RuntimeError: CUDA out of memory. Tried to allocate 742.00 MiB (GPU 0; 3.00 GiB total capacity; 1.02 GiB already allocated; 635.55 MiB free; 1.33 GiB reserved in total by PyTorch)

In [18]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()

In [19]:
main(config)

|train| = 118314 |valid| = 29579
|vocab| = 12957 |classes| = 2
RNNClassifier(
  (emb): Embedding(12957, 256)
  (rnn): LSTM(256, 512, num_layers=4, batch_first=True, dropout=0.3, bidirectional=True)
  (generator): Linear(in_features=1024, out_features=2, bias=True)
  (activation): LogSoftmax(dim=-1)
)

Epoch [1/2]:   1%|      | 14/1849 [00:02<05:53,  5.19it/s, accuracy=0.419, loss=0.675, |g_param|=0.308, |param|=1.83e+3]Current run is terminating due to exception: CUDA out of memory. Tried to allocate 726.00 MiB (GPU 0; 3.00 GiB total capacity; 1.03 GiB already allocated; 635.55 MiB free; 1.33 GiB reserved in total by PyTorch)
Engine run is terminating due to exception: CUDA out of memory. Tried to allocate 726.00 MiB (GPU 0; 3.00 GiB total capacity; 1.03 GiB already allocated; 635.55 MiB free; 1.33 GiB reserved in total by PyTorch)


RuntimeError: CUDA out of memory. Tried to allocate 726.00 MiB (GPU 0; 3.00 GiB total capacity; 1.03 GiB already allocated; 635.55 MiB free; 1.33 GiB reserved in total by PyTorch)

In [5]:
import torch
import math


dtype = torch.float
# device = torch.device("cpu")
device = torch.device("cuda:0") # Uncomment this to run on GPU

# Create random input and output data
x = torch.linspace(-math.pi, math.pi, 2000, device=device, dtype=dtype)
y = torch.sin(x)

# Randomly initialize weights
a = torch.randn((), device=device, dtype=dtype)
b = torch.randn((), device=device, dtype=dtype)
c = torch.randn((), device=device, dtype=dtype)
d = torch.randn((), device=device, dtype=dtype)

learning_rate = 1e-6
for t in range(2000):
    # Forward pass: compute predicted y
    y_pred = a + b * x + c * x ** 2 + d * x ** 3

    # Compute and print loss
    loss = (y_pred - y).pow(2).sum().item()
    if t % 100 == 99:
        print(t, loss)

    # Backprop to compute gradients of a, b, c, d with respect to loss
    grad_y_pred = 2.0 * (y_pred - y)
    grad_a = grad_y_pred.sum()
    grad_b = (grad_y_pred * x).sum()
    grad_c = (grad_y_pred * x ** 2).sum()
    grad_d = (grad_y_pred * x ** 3).sum()

    # Update weights using gradient descent
    a -= learning_rate * grad_a
    b -= learning_rate * grad_b
    c -= learning_rate * grad_c
    d -= learning_rate * grad_d


print(f'Result: y = {a.item()} + {b.item()} x + {c.item()} x^2 + {d.item()} x^3')

99 286.8958740234375
199 194.43716430664062
299 132.79104614257812
399 91.66726684570312
499 64.21915435791016
599 45.888633728027344
699 33.63977813720703
799 25.449932098388672
899 19.97038459777832
999 16.30175018310547
1099 13.843850135803223
1199 12.195928573608398
1299 11.090190887451172
1399 10.347681045532227
1499 9.848665237426758
1599 9.513006210327148
1699 9.28702449798584
1799 9.134754180908203
1899 9.032052993774414
1999 8.962711334228516
Result: y = -0.007677676156163216 + 0.8473697304725647 x + 0.0013245270820334554 x^2 + -0.091997429728508 x^3


In [45]:
import torch
torch.version

<module 'torch.version' from 'C:\\Users\\Lukaid\\AppData\\Local\\Programs\\Python\\Python39\\lib\\site-packages\\torch\\version.py'>

In [4]:
# import torch
print(torch.__version__)
print(torch.version.cuda)

1.8.1+cu111
11.1


In [10]:
import torchtext

ImportError: DLL load failed while importing _torchtext: 지정된 프로시저를 찾을 수 없습니다.

In [3]:
import torch

AttributeError: module 'torch.cuda' has no attribute '__version__'

In [1]:
from konlpy.tag import Mecab

ModuleNotFoundError: No module named 'konlpy'

In [2]:
from gensim.models import FastText

ModuleNotFoundError: No module named 'gensim'

In [3]:
import mecab

ModuleNotFoundError: No module named 'mecab'

In [59]:
import pandas as pd

test = pd.read_table('data/review.sorted.uniq.refined.tok.shuf.train.tsv', encoding= 'utf-8', header=None)

In [61]:
print('positive : ', sum(test.iloc[:, 0] == 'positive'))
print('negative : ', sum(test.iloc[:, 0] == 'negative'))

positive :  64978
negative :  64979


In [49]:
print('positive : ', sum(test.iloc[:, 0] == 'positive'))
print('negative : ', sum(test.iloc[:, 0] == 'negative'))

positive :  82914
negative :  64979


In [50]:
test

,0,1
0,negative,너무 얇 아 여 ~~ 이걸 어디 에 써야 는지 ㅠ
1,positive,사은품 은 보라색 왔 네요 핑크 만 빼 고 왔음 했 는데 다행 다행 ^^
2,positive,이 소재 가 요가 하 기 에 제일 좋 은 거 같 아요 예전 에 뚜꺼운 거 잘 못 샀...
3,negative,배송 3 주 걸림 . 블라인드 가 다 거기 서 거기 인 듯 함 . 뭔가 뛰어난 건 ...
4,positive,생각 보다 깨끗 하 네요 . . 잃어버릴 생각 으로 구매 했 는데 . . 아까울 듯...
...,...,...
147888,positive,고민 안 하 셔도 될 것 같 습니다 최고 예요 이 가격 에 이런 재품 없 습니다
147889,negative,비닐 이 액정 안 에 들어가 있 는데 제조 과정 에서 들어간 거 같 아요 . 뺄 수...
147890,negative,1 + 1 이 란 말 에 속 지 맙 시 다 . 리뷰 나 sns 올리 고 사진 첨부 ...
147891,positive,이번 에 트럭 에 바꿀 블 박 을 찾 다가 가 성비 좋 다는 파인 뷰 x 5 new...


In [52]:
test = test.drop(3, 0)
test

,0,1
1,positive,사은품 은 보라색 왔 네요 핑크 만 빼 고 왔음 했 는데 다행 다행 ^^
2,positive,이 소재 가 요가 하 기 에 제일 좋 은 거 같 아요 예전 에 뚜꺼운 거 잘 못 샀...
4,positive,생각 보다 깨끗 하 네요 . . 잃어버릴 생각 으로 구매 했 는데 . . 아까울 듯...
5,negative,2013 년 제품 입니다 .
6,positive,워낙 유명 한 제품 이 라 다 아 시 겠 지만 저렴 하 게 샀어요
...,...,...
147888,positive,고민 안 하 셔도 될 것 같 습니다 최고 예요 이 가격 에 이런 재품 없 습니다
147889,negative,비닐 이 액정 안 에 들어가 있 는데 제조 과정 에서 들어간 거 같 아요 . 뺄 수...
147890,negative,1 + 1 이 란 말 에 속 지 맙 시 다 . 리뷰 나 sns 올리 고 사진 첨부 ...
147891,positive,이번 에 트럭 에 바꿀 블 박 을 찾 다가 가 성비 좋 다는 파인 뷰 x 5 new...


In [53]:
test.iloc[3, 0]

'negative'

In [60]:
i = 0 
i_ = 0
while True:
    if test.iloc[i_, 0] == 'positive':
        test = test.drop(i, 0)
        i += 1
        print(sum(test.iloc[:, 0] == 'positive'))
        print(sum(test.iloc[:, 0] == 'negative'))
        if sum(test.iloc[:, 0] == 'positive') <= 64978:
            break
    else:
        i += 1
        i_ += 1

4
64979
66643
64979
66642
64979
66641
64979
66640
64979
66639
64979
66638
64979
66637
64979
66636
64979
66635
64979
66634
64979
66633
64979
66632
64979
66631
64979
66630
64979
66629
64979
66628
64979
66627
64979
66626
64979
66625
64979
66624
64979
66623
64979
66622
64979
66621
64979
66620
64979
66619
64979
66618
64979
66617
64979
66616
64979
66615
64979
66614
64979
66613
64979
66612
64979
66611
64979
66610
64979
66609
64979
66608
64979
66607
64979
66606
64979
66605
64979
66604
64979
66603
64979
66602
64979
66601
64979
66600
64979
66599
64979
66598
64979
66597
64979
66596
64979
66595
64979
66594
64979
66593
64979
66592
64979
66591
64979
66590
64979
66589
64979
66588
64979
66587
64979
66586
64979
66585
64979
66584
64979
66583
64979
66582
64979
66581
64979
66580
64979
66579
64979
66578
64979
66577
64979
66576
64979
66575
64979
66574
64979
66573
64979
66572
64979
66571
64979
66570
64979
66569
64979
66568
64979
66567
64979
66566
64979
66565
64979
66564
64979
66563
64979
66562
64979
66561
64

In [63]:
test = test.sample(frac=1).reset_index(drop=True)

In [64]:
test.to_csv('review.sorted.uniq.refined.tok.shuf.train.balanced.tsv', index=False, header=None, sep="\t")